In [1]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt
import scipy.stats as st
import sys
sys.path.insert(0, '/data2/duren_lab/idebnat/palmetto/Modest/Chrom_spatial/Bench_methods/SpaGE/')
from SpaGE.main import SpaGE

In [2]:
import scanpy as sc
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="numpy.core.fromnumeric")

In [11]:
import psutil
import time
import os

In [ ]:
rna=sc.read_h5ad("./rnap21.h5ad")
atac=sc.read_h5ad(".atacp21.h5ad")

In [5]:
RNA_data=pd.DataFrame(rna.X, columns=rna.var_names, index=rna.obs_names)
ATAC_data=pd.DataFrame(atac.X, columns=atac.var_names, index=atac.obs_names)

# Normalization
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1)
RNA_data = RNA_data.apply(Log_Norm_cpm,axis=0)
ATAC_data = ATAC_data.apply(Log_Norm_cpm,axis=0)

In [6]:
sc_multi=pd.concat([RNA_data, ATAC_data], axis=1)

In [ ]:
spatac=sc.read_h5ad("./spatacp21.h5ad")
SPATAC_data=pd.DataFrame(spatac.X, columns=spatac.var_names, index=spatac.obs_names)

In [ ]:
sprna=sc.read_h5ad("./sprnap21.h5ad")
SPRNA_data=pd.DataFrame(sprna.X, columns=sprna.var_names, index=sprna.obs_names)
# Normalization
cell_count = np.sum(SPRNA_data,axis=0)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)
SPRNA_data = SPRNA_data.apply(Log_Norm_spatial,axis=0)

In [12]:
Gene_set=ATAC_data.columns.values
Correlations = pd.Series(index = Gene_set)

#computation time and memory usage
process = psutil.Process(os.getpid())
# Record initial memory usage
memory_before = process.memory_info().rss
# Record start time
start_time = time.time()

Imp_Genes = SpaGE(SPRNA_data,sc_multi,n_pv=30, genes_to_predict = Gene_set)

# Record end time
end_time = time.time()
# Record final memory usage
memory_after = process.memory_info().rss
# Compute results
memory_used = (memory_after - memory_before) / 1024**2  # Convert to MB
elapsed_time = end_time - start_time  # Time in seconds

print(f"Elapsed Time: {elapsed_time:.4f} seconds")
print(f"Memory Used: {memory_used:.4f} MB")

Elapsed Time: 21.3713 seconds
Memory Used: 149.7539 MB


In [9]:
for i in Gene_set:
    Correlations[i] = st.pearsonr(SPATAC_data[i],Imp_Genes[i])[0]
    

In [11]:
px=pd.DataFrame(Imp_Genes)
px.columns = ATAC_data.columns.values
px.index=SPRNA_data.index.values
px.to_csv("SPAGE_P21_data.csv")